In [ ]:
import pandas as pd
import re                       # Import Regular Expression (remove HTML tags)
import string                   # Import Punctuation 
from textblob import TextBlob    # Import this Library to Handle the Spelling Issue
import nltk
from nltk.corpus import stopwords    #  NLTK library to remove Stopwords
# from nltk.stem import WordNetLemmatizer
import emoji                        # for translating symbol to text
import spacy                        # for tokenization
from nltk.stem.porter import PorterStemmer   # for stemming
from nltk.stem import WordNetLemmatizer      # for lemma
from sklearn.model_selection import train_test_split
from chat_words import chat_word                #for translate slang of charts to text
from autocorrect import Speller                 # for Spelling Correction


In [2]:
df = pd.read_csv(r'C:\Users\Admin\WORK\Project_CV\Model_NLP_sentiment\data\IMDB Dataset.csv')    # insert path to your data

In [4]:
df.head()                                                # check dataframe 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
#Access the corpus and target variables
x = df.review
y = df.sentiment                                                                             #.replace({1:'Negative', 2:'Positive'})

# train test splitting
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.0025, random_state=0)
X_test.shape

(125,)

In [26]:
# function for preporation data
def preprocessing (text):
    text = text.lower()                                  # LoweCasing Text

    pattern_1 = re.compile('<.*?>')                      # constant using one regular expression
    pattern_2 = re.sub(pattern_1, r'', text)             # changes ('<.*?>') to gap " "

    pattern_3 = re.compile(r'https?://\S+|www\.\S+')     #  Remove URLs from Text or Whole Corpus.
    pattern_4 = pattern_3.sub(r'', pattern_2)

    punc = string.punctuation                            # Remove punctuation
    pattern_5 = pattern_4.translate(str.maketrans('', '', punc))
    
    new_text = []
    for i in pattern_5.split():
        if i.upper() in chat_word:
            new_text.append(chat_word[i.upper()])
        else:
            new_text.append(i)
    pattern_6 = " ".join(new_text)
    new_text.clear()

    spell = Speller(lang='en')                             # Spelling Correction
    pattern_7 = spell(pattern_6)

    stopword = stopwords.words('english')                   # Handling StopWords
    for word in pattern_7.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
    pattern_8 = new_text[:]
    pattern_9 = " ".join(pattern_8)

    pattern_10 = emoji.demojize(pattern_9)                # Handling Emojies 

    stemmer = PorterStemmer()                              #Stemming
    pattern_11 = " ".join([stemmer.stem(word) for
                            word in pattern_10.split()])                    
    
    wordnet_lemmatizer = WordNetLemmatizer()              #Lemmatization
    pattern_12 = wordnet_lemmatizer.lemmatize(pattern_11,pos='v')
    return pattern_12













   

In [7]:
X_test = X_test.apply(preprocessing)
X_test


11841    john cassavetes    run   law     bottom   heap...
19602         movie  lame      movie   unnecessary   ne...
45519    well     ether waters   7yearly spammy davis j...
25747     find alan jacobs review  accurate concerning ...
42642     movie  simply awesome    hilarious although  ...
                               ...                        
30995       greatest movie ever    written     ever see...
3336      movie changed  life hogan performance  nothin...
5665       first saw  premiere episode  farscape    ide...
28316    home room deals   columbinelike highschool sho...
12359     watched  hoping  find  something  didnt know ...
Name: review, Length: 125, dtype: object

In [8]:
check_data = {
    "review": ["@lapcat need to send 'em to my accountant tomorrow. oddly, i wasn't even referring to my taxes. those are supporting evidence, though. ",
                "<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='http://google.com'>download</a></p></body></html>",
                 'Check out my notebook https://www.kaggle.com/campusx/notebook8223fc1', 'IMHO he is the best', 'FYI Islamabad is the capital of Pakistan',
                 'ceertain conditionas duriing seveal ggenerations aree moodified in the saame maner', 'probably my all-time favorite movie, a story of selflessness,'
                 ' sacrifice and dedication to a noble cause', "Loved the movie. It was 😘", "walk walks walking walked",
                 "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
]
    }

# Convert to DataFrame
df_check = pd.DataFrame(check_data)

# Print the DataFrame
prov = df_check['review']
prov


0    @lapcat need to send 'em to my accountant tomo...
1    <html><body><p> Movie 1</p><p> Actor - Aamir K...
2    Check out my notebook https://www.kaggle.com/c...
3                                  IMHO he is the best
4             FYI Islamabad is the capital of Pakistan
5    ceertain conditionas duriing seveal ggeneratio...
6    probably my all-time favorite movie, a story o...
7                            Loved the movie. It was 😘
8                            walk walks walking walked
9    He was running and eating at same time. He has...
Name: review, dtype: object

In [27]:
prov = prov.apply(preprocessing)
prov


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\Admin/nltk_data'
    - 'c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python313\\nltk_data'
    - 'c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python313\\share\\nltk_data'
    - 'c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python313\\lib\\nltk_data'
    - 'C:\\Users\\Admin\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
#Tokenization
nlp = spacy.load('en_core_web_sm')                     # the English language model 'en_core_web_sm
df['review'] = df['review'].apply(nlp)


I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.

In [14]:
print(df['review'][1:3])

1    (A, wonderful, little, production, ., <, br, /...
2    (I, thought, this, was, a, wonderful, way, to,...
Name: review, dtype: object


In [9]:
#Lemmatization

wordnet_lemmatizer = WordNetLemmatizer()


df['review'] = df['review'].apply(wordnet_lemmatizer.lemmatize(df['review'],pos='v'))
df['review'][1]



LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\Admin/nltk_data'
    - 'c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python313\\nltk_data'
    - 'c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python313\\share\\nltk_data'
    - 'c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python313\\lib\\nltk_data'
    - 'C:\\Users\\Admin\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
